In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_file = './20000_vectors_drags.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_19992,dim_19993,dim_19994,dim_19995,dim_19996,dim_19997,dim_19998,dim_19999,dim_20000,drag
61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.375
354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.374
358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.435
275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.437
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.367


In [2]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    351.000000
mean       0.398513
std        0.060013
min        0.278000
25%        0.353000
50%        0.394000
75%        0.435000
max        0.598000
Name: drag, dtype: float64


In [3]:
import os
save_path = './agModels-20000'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, presets='best_quality', num_bag_folds=0, num_bag_sets=1)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=0, num_bag_sets=1
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "./agModels-20000/"
AutoGluon Version:  0.7.0
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #51-Ubuntu SMP Mon Jul 4 06:41:22 UTC 2022
Train Data Rows:    351
Train Data Columns: 20000
Label Column: drag
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (0.598, 0.278, 0.39851, 0.06001)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predi

In [4]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_19991,dim_19992,dim_19993,dim_19994,dim_19995,dim_19996,dim_19997,dim_19998,dim_19999,dim_20000
46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
%%capture log_output
%config InlineBackend.figure_format = 'retina'
%config Application.log_level = 'DEBUG'
%config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

with open('./output_20000.log', 'w') as f:
    f.write(log_output.stdout)

Evaluation: root_mean_squared_error on test data: -0.04472043501814541
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -0.04472043501814541,
    "mean_squared_error": -0.0019999173082121662,
    "mean_absolute_error": -0.030963519275188447,
    "r2": 0.3198973710981401,
    "pearsonr": 0.5678180015967901,
    "median_absolute_error": -0.02152097260951996
}


In [7]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('int', ['bool']) : 8832 | ['dim_150', 'dim_170', 'dim_171', 'dim_189', 'dim_191', ...]


In [8]:
train_data_pred = predictor.predict(train_data, model='WeightedEnsemble_L2')
for item in train_data_pred:
    print(item)

0.3758005201816559
0.38704121112823486
0.4321252703666687
0.4150639772415161
0.3685636818408966
0.3649337887763977
0.4200798273086548
0.35727375745773315
0.345221608877182
0.3455789387226105
0.36319613456726074
0.4947512149810791
0.473291277885437
0.37871116399765015
0.42338865995407104
0.44608357548713684
0.3198503255844116
0.3983652591705322
0.4932929277420044
0.3626866638660431
0.41345933079719543
0.4467672109603882
0.3539583086967468
0.4435587525367737
0.4053468406200409
0.4337852895259857
0.3994835317134857
0.41477465629577637
0.3321647644042969
0.34673723578453064
0.45225828886032104
0.42307785153388977
0.47660595178604126
0.4436551332473755
0.35521960258483887
0.38124650716781616
0.3930683135986328
0.42228594422340393
0.3653751611709595
0.3361767828464508
0.5235490798950195
0.3792968690395355
0.3424113690853119
0.30952298641204834
0.3472285568714142
0.4263523817062378
0.4061618447303772
0.39948365092277527
0.34427276253700256
0.40362533926963806
0.38663753867149353
0.35612359642

In [9]:
test_data_pred = predictor.predict(test_data, model='WeightedEnsemble_L2')
for item in test_data_pred:
    print(item)

0.36304333806037903
0.3936271071434021
0.3734660744667053
0.49298998713493347
0.3825214207172394
0.41101422905921936
0.43216463923454285
0.429278165102005
0.4113698899745941
0.3352794945240021
0.36495524644851685
0.3972771167755127
0.3774583637714386
0.4288546144962311
0.42549988627433777
0.4449315369129181
0.39592140913009644
0.4049135446548462
0.3743479549884796
0.36385709047317505
0.43034127354621887
0.4104434549808502
0.3677077889442444
0.38110509514808655
0.46038633584976196
0.4091483950614929
0.41752150654792786
0.37650689482688904
0.4032394289970398
0.39421889185905457
0.439737468957901
0.3582433760166168
0.387124240398407
0.39961519837379456
0.4064837694168091
0.37005409598350525
0.3810204863548279
0.4062502086162567
0.3735235929489136
0.3979983925819397
0.47102710604667664
0.3489489257335663
0.44411006569862366
0.3645387887954712
0.4131363034248352
0.4102960526943207
0.4248524010181427
0.3924502432346344
0.4001351296901703
0.3608521521091461
0.3757493495941162
0.36398321390151